In [1]:
import pandas as pd

In [2]:
df_local = pd.read_parquet('data/datalake/daily/2025/07/2025-07-30-data.parquet')
df_local

,Fecha,Estado,Ciudad,Tipo,Canal,Precio
0,2025-07-30,Aguascalientes,Aguascalientes,Pasteurizada,Tiendas,27.40
1,2025-07-30,Baja California,Mexicali,Pasteurizada,Tiendas,29.30
2,2025-07-30,Baja California,Tijuana,Pasteurizada,Tiendas,NaN
3,2025-07-30,Baja California Sur,La Paz,Pasteurizada,Tiendas,26.00
4,2025-07-30,Campeche,Campeche,Pasteurizada,Tiendas,NaN
...,...,...,...,...,...,...
211,2025-07-30,Veracruz,Coatzacoalcos,Ultrapasteurizada,Autoservicios,34.00
212,2025-07-30,Veracruz,Xalapa,Ultrapasteurizada,Autoservicios,35.58
213,2025-07-30,Veracruz,Veracruz,Ultrapasteurizada,Autoservicios,36.10
214,2025-07-30,Yucatán,Mérida,Ultrapasteurizada,Autoservicios,34.42


In [3]:
s3_path = "s3://mlops-milk-datalake/daily/2025/07/2025-07-30-data.parquet"
df_s3 = pd.read_parquet(s3_path)
df_s3

,Fecha,Estado,Ciudad,Tipo,Canal,Precio
0,2025-07-30,Aguascalientes,Aguascalientes,Pasteurizada,Tiendas,27.40
1,2025-07-30,Baja California,Mexicali,Pasteurizada,Tiendas,29.30
2,2025-07-30,Baja California,Tijuana,Pasteurizada,Tiendas,NaN
3,2025-07-30,Baja California Sur,La Paz,Pasteurizada,Tiendas,26.00
4,2025-07-30,Campeche,Campeche,Pasteurizada,Tiendas,NaN
...,...,...,...,...,...,...
211,2025-07-30,Veracruz,Coatzacoalcos,Ultrapasteurizada,Autoservicios,34.00
212,2025-07-30,Veracruz,Xalapa,Ultrapasteurizada,Autoservicios,35.58
213,2025-07-30,Veracruz,Veracruz,Ultrapasteurizada,Autoservicios,36.10
214,2025-07-30,Yucatán,Mérida,Ultrapasteurizada,Autoservicios,34.42


In [4]:
df_ref = pd.read_parquet('../data/datalake/daily/2025/07/2025-07-28-data.parquet')
df_ref

,Fecha,Estado,Ciudad,Tipo,Canal,Precio
0,2025-07-28,Aguascalientes,Aguascalientes,Pasteurizada,Tiendas,27.40
1,2025-07-28,Baja California,Mexicali,Pasteurizada,Tiendas,29.30
2,2025-07-28,Baja California,Tijuana,Pasteurizada,Tiendas,NaN
3,2025-07-28,Baja California Sur,La Paz,Pasteurizada,Tiendas,26.00
4,2025-07-28,Campeche,Campeche,Pasteurizada,Tiendas,NaN
...,...,...,...,...,...,...
211,2025-07-28,Veracruz,Coatzacoalcos,Ultrapasteurizada,Autoservicios,34.13
212,2025-07-28,Veracruz,Xalapa,Ultrapasteurizada,Autoservicios,35.58
213,2025-07-28,Veracruz,Veracruz,Ultrapasteurizada,Autoservicios,35.35
214,2025-07-28,Yucatán,Mérida,Ultrapasteurizada,Autoservicios,34.55


In [ ]:
df2

In [ ]:
df.tail()

In [ ]:
filtered 

In [ ]:
from datetime import datetime
import boto3
import requests

# --- Configuration ---
BUCKET_NAME = "mlops-milk-datalake"
S3_PREFIX = "daily"
BASE_URL = "https://www.economia-sniim.gob.mx/SNIIM-Archivosfuente/Comentarios/Otros"

# --- Today's date ---
today = datetime.today()
excel_filename = f"Leche{today.strftime('%d%m%Y')}.xlsx"
parquet_filename = f"{today.strftime('%Y-%m-%d')}-data.parquet"
s3_key = f"{S3_PREFIX}/{today.year}/{today.strftime('%m')}/{parquet_filename}"

print(f"🔍 Expected Excel file: {excel_filename}")
print(f"🔍 Expected S3 key: {s3_key}")

# --- Step 1: Check if file already exists in S3 ---
s3 = boto3.client("s3")

def s3_file_exists(bucket: str, key: str) -> bool:
    try:
        s3.head_object(Bucket=bucket, Key=key)
        print("✅ File already exists in S3.")
        return True
    except s3.exceptions.ClientError as e:
        if e.response["Error"]["Code"] == "404":
            print("🆕 File not found in S3.")
            return False
        else:
            raise

# --- Step 2: Check if Excel file is published online ---
def remote_file_exists(url: str) -> bool:
    response = requests.head(url)
    if response.status_code == 200:
        print("✅ Excel file is available online.")
        return True
    elif response.status_code == 404:
        print("❌ Excel file not available yet.")
        return False
    else:
        print(f"⚠️ Unexpected response: {response.status_code}")
        return False

# --- Decision logic ---
file_in_s3 = s3_file_exists(BUCKET_NAME, s3_key)
file_online = remote_file_exists(f"{BASE_URL}/{excel_filename}")

should_process = not file_in_s3 and file_online

print(f"\n🚦 Should process today’s file? {'✅ YES' if should_process else '❌ NO'}")


In [ ]:
import pandas as pd
import requests
from datetime import datetime
from pathlib import Path
import boto3
import io

# --- Constants ---
BASE_URL = "https://www.economia-sniim.gob.mx/SNIIM-Archivosfuente/Comentarios/Otros"
today = datetime.today()
excel_filename = f"Leche{today.strftime('%d%m%Y')}.xlsx"
local_excel_path = f"data/raw/{excel_filename}"
parquet_filename = f"{today.strftime('%Y-%m-%d')}-data.parquet"
s3_key = f"daily/{today.year}/{today.strftime('%m')}/{parquet_filename}"
bucket = "mlops-milk-datalake"

# --- Download the Excel file ---
url = f"{BASE_URL}/{excel_filename}"
print(f"📥 Downloading: {url}")
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"Failed to download Excel file: {response.status_code}")
    
# Save locally
Path("data/raw").mkdir(parents=True, exist_ok=True)
with open(local_excel_path, "wb") as f:
    f.write(response.content)

print(f"✅ File saved to: {local_excel_path}")

# --- Parse Excel file ---
def parse_excel_to_df(path):
    xls = pd.ExcelFile(path)
    data_final = []
    for sheet in xls.sheet_names:
        df = xls.parse(sheet, skiprows=6)
        if "Estado" not in df.columns:
            continue
        df = df.iloc[:, :6]
        df.columns = ["Estado", "Ciudad", "Pasteurizada_Tiendas", "Pasteurizada_Autoservicios", "Ultrapasteurizada_Tiendas", "Ultrapasteurizada_Autoservicios"]
        df = df.dropna(how="all")

        df_long = df.melt(id_vars=["Estado", "Ciudad"], 
                          value_vars=["Pasteurizada_Tiendas", "Pasteurizada_Autoservicios", 
                                      "Ultrapasteurizada_Tiendas", "Ultrapasteurizada_Autoservicios"],
                          var_name="Tipo_Canal", value_name="Precio")

        df_long["Tipo"] = df_long["Tipo_Canal"].apply(lambda x: x.split("_")[0])
        df_long["Canal"] = df_long["Tipo_Canal"].apply(lambda x: x.split("_")[1])
        df_long.drop(columns=["Tipo_Canal"], inplace=True)
        df_long["Fecha"] = today.date()
        df_long["Precio"] = pd.to_numeric(df_long["Precio"], errors="coerce")

        data_final.append(df_long)

    if not data_final:
        raise ValueError("No valid sheets found in Excel file.")

    df_final = pd.concat(data_final, ignore_index=True)

    # Remove rows where Ciudad includes "promedio"
    df_final = df_final[~df_final["Ciudad"].str.lower().str.contains("promedio", na=False)]

    return df_final

df = parse_excel_to_df(local_excel_path)

# --- Debug: Show number of rows and dates ---
print(f"✅ Parsed rows: {len(df)}")
print(f"🗓️  Distinct dates found: {df['Fecha'].unique()}")

# --- Filter for today's date ---
df_today = df[df["Fecha"] == today.date()].copy()

if df_today.empty:
    raise ValueError("❌ No rows found for today's date.")

# --- Save locally ---
Path("data/datalake/daily")\
    .joinpath(str(today.year), today.strftime("%m"), "")\
    .mkdir(parents=True, exist_ok=True)

local_parquet_path = f"data/datalake/daily/{today.year}/{today.strftime('%m')}/{parquet_filename}"
df_today.to_parquet(local_parquet_path, index=False)
print(f"✅ Parquet file saved locally: {local_parquet_path}")

# --- Upload to S3 ---
s3 = boto3.client("s3")
with open(local_parquet_path, "rb") as f:
    s3.upload_fileobj(f, bucket, s3_key)
print(f"✅ Uploaded to S3: s3://{bucket}/{s3_key}")


este anterior no funciona por que incluye todas las filas en el archivo del día

In [ ]:
from datetime import datetime
import pandas as pd
from pathlib import Path
import boto3
import requests
import re

# --- Config ---
today = datetime.today().date()
EXCEL_FILENAME = f"Leche{today.strftime('%d%m%Y')}.xlsx"
EXCEL_URL = f"https://www.economia-sniim.gob.mx/SNIIM-Archivosfuente/Comentarios/Otros/{EXCEL_FILENAME}"
EXCEL_PATH = f"data/raw/{EXCEL_FILENAME}"
PARQUET_PATH = f"data/datalake/daily/{today.year}/{today.strftime('%m')}/{today.strftime('%Y-%m-%d')}-data.parquet"
S3_BUCKET = "mlops-milk-datalake"
S3_KEY = f"daily/{today.year}/{today.strftime('%m')}/{today.strftime('%Y-%m-%d')}-data.parquet"

# --- Utilidades ---
meses_es_a_en = {
    'enero': 'January', 'febrero': 'February', 'marzo': 'March',
    'abril': 'April', 'mayo': 'May', 'junio': 'June',
    'julio': 'July', 'agosto': 'August', 'septiembre': 'September',
    'octubre': 'October', 'noviembre': 'November', 'diciembre': 'December'
}

def download_excel():
    Path("data/raw").mkdir(parents=True, exist_ok=True)
    response = requests.get(EXCEL_URL)
    if response.status_code != 200:
        raise Exception(f"❌ Failed to download Excel file: {response.status_code}")
    with open(EXCEL_PATH, "wb") as f:
        f.write(response.content)
    print(f"✅ File downloaded: {EXCEL_PATH}")

def parse_excel_to_df(path=EXCEL_PATH):
    df = pd.read_excel(path, header=None)
    data_final = []

    for i, row in df.iterrows():
        if row.astype(str).str.contains("Precio promedio al consumidor por litro", case=False, na=False).any():
            row_fecha = df.iloc[i + 1]
            fecha_raw = next((str(cell) for cell in row_fecha if isinstance(cell, str) and "de" in cell), None)
            if not fecha_raw:
                continue

            match = re.search(r"(\d{1,2}) de (\w+) de (\d{4})", fecha_raw, re.IGNORECASE)
            if not match:
                continue
            dia, mes, año = match.groups()
            mes_en = meses_es_a_en.get(mes.lower())
            if not mes_en:
                continue
            fecha = pd.to_datetime(f"{dia} {mes_en} {año}", dayfirst=True).date()

            enc2 = df.iloc[i + 3, 2:6].values
            columnas = ["Estado", "Ciudad"]
            for j in range(4):
                tipo = "Pasteurizada" if j < 2 else "Ultrapasteurizada"
                canal = enc2[j]
                columnas.append(f"{tipo}_{canal}")

            df_bloque = df.iloc[i + 4:, 0:6].copy()
            df_bloque.columns = columnas
            df_bloque[['Estado', 'Ciudad']] = df_bloque[['Estado', 'Ciudad']].ffill()

            stop_idx = None
            for k, row_k in df_bloque.iterrows():
                celda = str(row_k[0]).strip().lower()
                if "promedio" in celda or "fuente" in celda or "sniim" in celda or celda == "estado" or celda == "nan":
                    stop_idx = k
                    break
            if stop_idx:
                df_bloque = df_bloque.loc[:stop_idx - 1]

            df_bloque["Fecha"] = fecha
            df_long = df_bloque.melt(id_vars=["Fecha", "Estado", "Ciudad"],
                                     var_name="Tipo_Canal", value_name="Precio")

            df_long["Tipo"] = df_long["Tipo_Canal"].str.split("_").str[0]
            df_long["Canal"] = df_long["Tipo_Canal"].str.split("_").str[1]
            df_long = df_long[["Fecha", "Estado", "Ciudad", "Tipo", "Canal", "Precio"]]
            df_long["Precio"] = pd.to_numeric(df_long["Precio"], errors="coerce")

            data_final.append(df_long)

    df_final = pd.concat(data_final, ignore_index=True)
    df_final = df_final[~df_final["Ciudad"].str.lower().str.contains("promedio", na=False)]
    return df_final

def save_to_parquet(df):
    Path(PARQUET_PATH).parent.mkdir(parents=True, exist_ok=True)
    df.to_parquet(PARQUET_PATH, index=False)
    print(f"✅ Parquet file saved locally: {PARQUET_PATH}")

def upload_to_s3(local_path, bucket, key):
    s3 = boto3.client("s3")
    with open(local_path, "rb") as f:
        s3.upload_fileobj(f, bucket, key)
    print(f"✅ Uploaded to S3: s3://{bucket}/{key}")

# --- Pipeline ---
download_excel()
df_all = parse_excel_to_df()
df_today = df_all[df_all["Fecha"] == today].copy()
if df_today.empty:
    raise ValueError("❌ No rows found for today's date.")
save_to_parquet(df_today)
upload_to_s3(PARQUET_PATH, S3_BUCKET, S3_KEY)
